In [ ]:
from keras.applications import InceptionV3
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')

input = Input(shape=(200, 200, 3))
model = InceptionV3(input_tensor=input, include_top=False, weights='imagenet', pooling='max')

for layer in model.layers:
  layer.trainable = False

input_image_size = (200, 200)

x = model.output
x = Dense(1024, name='fully', init='uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512, init='uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(101, activation='softmax', name='softmax')(x)
model = Model(model.input, x)

model.summary()

train_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/caltech101/caltech101',
    target_size=input_image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/caltech101/caltech101',
    target_size=input_image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

model.compile(
    optimizer=optimizers.adam(),
    loss='categorical_crossentropy',
    metrics=['acc'])

early_stopping = EarlyStopping(patience=20, mode='auto', monitor='val_acc')
hist = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples // batch_size,
                              validation_data = validation_generator,
                              epochs=100,
                              callbacks=[early_stopping])

import matplotlib
import matplotlib.pyplot as plt
# %matplotlib nbagg

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'g', label='validation loss')
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'r', label='validation acc')

plt.show()